In [ ]:
# Setup notebook
import random
import numpy as np

# Load the autoreload extension
%load_ext autoreload

# Set autoreload to reload all modules every time a cell is executed
%autoreload 2

seed = 0
random.seed(seed)
np.random.seed(seed)

In [ ]:
from dataset.common import resolve_folds
from dataset.movie_lens import load_ml_100k_folds

data, k_fold = load_ml_100k_folds()
folds = resolve_folds(data, k_fold)

folds_without_index = [fold[1] for fold in folds]

In [ ]:
from evaluation.strategies import (
    MAEStrategy,
    RMSEStrategy,
    MicroAveragedRecallStrategy,
    MacroAveragedRecallStrategy,
    RecallAtKStrategy,
    MicroAveragedPrecisionStrategy,
    MacroAveragedPrecisionStrategy,
    PrecisionAtKStrategy,
    NDCGStrategy,
    F1ScoreStrategy,
    PredictionCoverageStrategy,
)

train_measures = [ ]

test_measures = [
    MAEStrategy(verbose=False),
    RMSEStrategy(verbose=False),
    MicroAveragedRecallStrategy(threshold=4.0),
    MacroAveragedRecallStrategy(threshold=4.0),
    RecallAtKStrategy(k=20, threshold=4.0),
    MicroAveragedPrecisionStrategy(threshold=4.0),
    MacroAveragedPrecisionStrategy(threshold=4.0),
    PrecisionAtKStrategy(k=20, threshold=4.0),
    NDCGStrategy(k=20, threshold=4.0),
    F1ScoreStrategy(k=20, threshold=4.0),
    PredictionCoverageStrategy(),
]


In [ ]:
from surprise.prediction_algorithms import KNNWithMeans
from evaluation.benchmark import GridSearch

parameters_grid = {
    "k": [1, 5, 10, 20, 30, 50, 100, 200],
    "min_k": [1, 10, 20, 30],
    "sim_options": [
        {"name": "pearson", "user_based": False},
        {"name": "cosine", "user_based": False},
        {"name": "msd", "user_based": False},
        {"name": "pearson_baseline", "user_based": False},
    ],
    'verbose': [False],
}

grid_search = GridSearch(
    KNNWithMeans,
    parameters_grid,
    test_measures,
    train_measures,
    max_workers=16,
)

best, ordering, raw = grid_search.fit(folds_without_index)

In [ ]:
import json

with open('IBKNN_grid_best.results', 'w') as file_object:
    json.dump(best, file_object, indent=4)

with open('IBKNN_grid_ordering.results', 'w') as file_object:
    json.dump(ordering, file_object, indent=4)

with open('IBKNN_grid_raw.results', 'w') as file_object:
    json.dump(raw, file_object, indent=4)
